In [ ]:
import datetime
import os
import sys
from dataclasses import dataclass

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_decision_forests as tfdf
from tensorflow import keras

In [ ]:
cwd = os.getcwd()